In [1]:
DB = {
    'Person': {
        1: {'first_name': 'Isaac', 'last_name': 'Newton', 'born': 1642, 'country_id': 1},
        2: {'first_name': 'Gottfried', 'last_name': 'von Leibniz', 'born': 1646, 'country_id': 5},
        3: {'first_name': 'Joseph', 'last_name': 'Fourier', 'born': 1768, 'country_id': 3},
        4: {'first_name': 'Bernhard', 'last_name': 'Riemann', 'born': 1826, 'country_id': 5},
        5: {'first_name': 'David', 'last_name': 'Hilbert', 'born': 1862 , 'country_id': 5},
        6: {'first_name': 'Srinivasa', 'last_name': 'Ramanujan', 'born': 1887, 'country_id': 4},
        7: {'first_name': 'John', 'last_name': 'von Neumann', 'born': 1903, 'country_id': 2},
        8: {'first_name': 'Andrew', 'last_name': 'Wiles', 'born': 1928, 'country_id': 6}
    },
    'Country': {
        1: {'name': 'United Kingdom', 'capital': 'London', 'continent': 'Europe'},
        2 :{'name': 'Hungary', 'capital': 'Budapest', 'continent': 'Europe'},
        3: {'name': 'France', 'capital': 'Paris', 'continent': 'Europe'},
        4: {'name': 'India', 'capital': 'New Delhi', 'continent': 'Asia'},
        5: {'name': 'Germany', 'capital': 'Berlin', 'continent': 'Europe'},
        6: {'name': 'USA', 'capital': 'Washington DC', 'continent': 'North America'}
        }
}

In [2]:
class Country:
    # not a very good example on how to get data from DB, if DB changes, we would need to update this as well
    def __init__(self, id_):
        if _id in DB['Country']:
            self._db_record = DB['Country'][id_]
        else:
            raise ValueError(f'Record not found (Country.id={id_})')

    @property
    def name(self):
        return self._db_record['name']
    
    @property
    def capital(self):
        return self._db_record['capital']
    
    @property
    def continent(self):
        return self._db_record['continent']

In [3]:
class DBRecord:
    def __init__(self, db_record_dict):
        # again, careful how you set a property on instances of this class
        # because we are overriding __setattr__ we cannot just use 
        # self._record = db_record_dict
        # this will call OUR version of `__setattr__`, which attempts to 
        # see if name is in _record - but _record does not exist yet, so it will
        # call __getattr__, which in turn tries to check if that is contained in _record
        # so, infinite recursion.
        # What we want to here is BYPASS our custom __setattr__ - so we'll use
        # the one in the superclass.
        super().__setattr__('_record', db_record_dict)    
        
    def __getattr__(self, name):
        # here we could write
        #     if name in self._record 
        # since this method should not get called
        # before _record as been created.
        # But just to be on the safe side, I'm still going to use super
        
        if name in super().__getattribute__('_record'):
            return self._record[name]
        else:
            raise AttributeError(f'Field name {name} does not exist.')

    def __setattr__(self, name, value):
        # and again here, we could write
        # if name in self._record, but I'm still going to use super
        if name in super().__getattribute__('_record'):
            super().__setattr__(name, value)
            # self._record[name] = value
        else:
            raise AttributeError(f'Field name {name} does not exist.')

In [4]:
class DBTable:
    def __init__(self, db, table_name):
        if table_name not in db:
            raise ValueError(f"The table {table_name} does not exist in the DB.")
        self._table_name = table_name
        self._table = db[table_name]

    @property
    def table_name(self):
        return self._table_name

    def __call__(self, record_id):
        if record_id not in self._table:
            raise ValueError(f"Specified if {record_id} does not exist in table {self.table_name}")
        return DBRecord(self._table[record_id])
        

In [5]:
table_person = DBTable(DB, "Person")
table_country = DBTable(DB, "Country")
p1 = table_person(1)

In [6]:
p1.__dict__

{'_record': {'first_name': 'Isaac',
  'last_name': 'Newton',
  'born': 1642,
  'country_id': 1}}

In [7]:
p1.first_name, p1.last_name

('Isaac', 'Newton')

In [8]:
c1 = table_country(p1.country_id)

In [9]:
c1.__dict__

{'_record': {'name': 'United Kingdom',
  'capital': 'London',
  'continent': 'Europe'}}

In [10]:
class DBRecord:
    def __init__(self, db_record_dict):
        super().__setattr__('_record', db_record_dict)    
        
    def __getattr__(self, name):
        if name in super().__getattribute__('_record'):
            return self._record[name]
        else:
            raise AttributeError(f'Field name {name} does not exist.')

    def __setattr__(self, name, value):
        if name in super().__getattribute__('_record'):
            super().__setattr__(name, value)
        else:
            raise AttributeError(f'Field name {name} does not exist.')

    @property
    def fields(self):
        record = super().__getattribute__("_record")  # use __getattribure__ to avoid error at __getattr__
        return tuple(record.keys())


In [11]:
t_person = DBTable(DB, "Person")
p1 = t_person(2)
type(p1)

__main__.DBRecord

In [12]:
p1.fields

('first_name', 'last_name', 'born', 'country_id')

In [13]:
p1.first_name

'Gottfried'

In [14]:
p1.first_name = "Gottfried van der Hoven"

In [15]:
p1.__dict__

{'_record': {'first_name': 'Gottfried',
  'last_name': 'von Leibniz',
  'born': 1646,
  'country_id': 5},
 'first_name': 'Gottfried van der Hoven'}